<a href="https://colab.research.google.com/github/juanstepanenko/Prueba-red/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import applications

def model(amountClasses):
    vgg=applications.vgg16.VGG16()
    cnn=Sequential()
    for layer in vgg.layers:
        cnn.add(layer)
    cnn.layers.pop()
    for layer in cnn.layers:
        layer.trainable=False
    cnn.add(Dense(amountClasses,activation='softmax'))
    
    return cnn

K.clear_session()

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

root_dir = "/content/gdrive/My Drive/TPS 5TO/Proyecto Final/Red Neuronal/Data/"

trainingData = root_dir + "Training"
validationData = root_dir + "Validation"

            
#trainingData = 'C:/Users/stepa/Desktop/Prueba-red/data/training'
#validationData = 'C:/Users/stepa/Desktop/Prueba-red/data/validation'

epochs=4 # Number of epochs ("itertions") of the the data set in training
length, height=224, 224 # Image size on pixels
batchSize=2 # Number of pictures to be processed in each step
steps=1000 # Number of batches proccesed in one iteration
validationSteps=20 # Number of steps at the end of each iteration    
conv1Filters=32 # Picture Depth
conv2Filters=64 # Picture Depth
filter1Size=(3, 3)
filter2Size=(2, 2)
poolSize=(2, 2) # Filter size used in the maxpooling
classes=3 # Number of problems
lr=0.0004 # Learning rate 


## Pictures preparation (pre-processing information)

datagenTraining=ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0, # This parameter rotates the diferent pictures (oj)
    zoom_range=0.2, 
    horizontal_flip=False) # Flips the picture

datagenValidation=ImageDataGenerator(rescale=1. / 255)

trainingImage=datagenTraining.flow_from_directory(
    trainingData,
    target_size=(length, height),
    batch_size=batchSize,
    class_mode="categorical") # We use this class mode when there are no classes/problem combinations

validationImage=datagenValidation.flow_from_directory(
    validationData,
    target_size=(length, height),
    batch_size=batchSize,
    class_mode="categorical")


# Network creation

cnn=model(classes)

cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])


cnn.fit_generator(
    trainingImage,
    steps_per_epoch=steps,
    epochs=epochs,
    validation_data=validationImage,
    validation_steps=validationSteps)


#target_dir = "/content/gdrive/My Drive/"
#base_dir = root_dir + 'fastai-v3/data/'

#target_dir = 'C:/Users/stepa/Desktop/Prueba-red/model'
#if not os.path.exists(target_dir):
  #os.mkdir(target_dir)
cnn.save("model.h5")
cnn.save_weights("weigths.h5")

from google.colab import files
#files.download("model.h5")
#files.download("weigths.h5")

Mounted at /content/gdrive
Found 17 images belonging to 3 classes.
Found 17 images belonging to 3 classes.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
553467904/553467096 [==============================] - 24s 0us/step
Epoch 1/4
1000/1000 [==============================] - 66s 66ms/step - loss: 1.0200 - acc: 0.5834 - val_loss: 0.9474 - val_acc: 0.6053
Epoch 2/4
1000/1000 [==============================] - 57s 57ms/step - loss: 0.9449 - acc: 0.5874 - val_loss: 0.9307 - val_acc: 0.6053
Epoch 3/4
1000/1000 [==============================] - 57s 57ms/step - loss: 0.9293 - acc: 0.5887 - val_loss: 0.9196 - val_acc: 0.6053
Epoch 4/4
1000/1000 [==============================] - 57s 57ms/step - loss: 0.9211 - acc: 0.5878 - val_loss: 0.8794 - val_acc: 0.6053


MessageError: ignored

In [37]:
import numpy as np

from keras.preprocessing.image import load_img, img_to_array
import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

length, height = 224, 224 # Same as in train.py
#model1 = "model.h5"
#weigths1 = "weigths.h5"

#with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
 # cnn = load_model(model)

#from keras.models import load_model

# Load files we already generated



#cnn.load_weights(weigths)

def predict(file): 
  image = load_img(file, target_size=(length, height))
  image = img_to_array(image)
  image = np.expand_dims(image, axis=0) # On axis 0 or first dimention we add a new dimention
  predictionArray = cnn.predict(image) # Format: [[1,0]]. On the first dimention it have a number 1 when the prediction is the correct one
  result = predictionArray[0] 
  answer = np.argmax(result) # Get index of the maximum result value
  if answer == 0:
    print("Prediction: Problem 1")
  elif answer == 1:
    print("Prediction: Problem 2")
  elif answer == 2:
    print("Prediction: Problem 3")

  return answer

testImage = validationData + "/Perdida en válvula/prueba.jpeg"
# How to predict:
predict(testImage) # File from validation

Prediction: Problem 3


2